<a href="https://cognitiveclass.ai"><img src = "https://ibm.box.com/shared/static/9gegpsmnsoo25ikkbl4qzlvlyjbgxs5x.png" width = 400> </a>

<h1 align=center><font size = 5>Learning FourSquare API with Python</font></h1>

## Introduction

In this lab, you will learn in details how to make calls to the Foursquare API for different purposes. You will learn how to construct a URL to send a request to the API to search for a specific type of venues, to explore a particular venue, to explore a Foursquare user, to explore a geographical location, and to get trending venues around a location. Also, you will learn how to use the visualization library, Folium, to visualize the results.

## Table of Contents

1. <a href="#item1">Foursquare API Search Function</a>
2. <a href="#item2">Explore a Given Venue</a>  
3. <a href="#item3">Explore a User</a>  
4. <a href="#item4">Foursquare API Explore Function</a>  
5. <a href="#item5">Get Trending Venues</a>  

### Import necessary Libraries

In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

#!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


### Define Foursquare Credentials and Version

##### Make sure that you have created a Foursquare developer account and have your credentials handy

In [2]:
CLIENT_ID = '534COXFXZVA22TB0B2PDNFEMGEHRNUXEHLSU5EN1BB0ICK1M' # your Foursquare ID
CLIENT_SECRET = 'XOUOKKHZCK1REWSLR3KYR4FQKCMWDWZVXNHSFW5BXNOQMB0V' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 534COXFXZVA22TB0B2PDNFEMGEHRNUXEHLSU5EN1BB0ICK1M
CLIENT_SECRET:XOUOKKHZCK1REWSLR3KYR4FQKCMWDWZVXNHSFW5BXNOQMB0V


#### Let's again assume that you are staying at the Conrad hotel. So let's start by converting the Contrad Hotel's address to its latitude and longitude coordinates.

In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>foursquare_agent</em>, as shown below.

In [3]:
address = '102 North End Ave, New York, NY'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

40.7149555 -74.0153365


<a id="item1"></a>

## 1. Search for a specific venue category
> `https://api.foursquare.com/v2/venues/`**search**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&ll=`**LATITUDE**`,`**LONGITUDE**`&v=`**VERSION**`&query=`**QUERY**`&radius=`**RADIUS**`&limit=`**LIMIT**

#### Now, let's assume that it is lunch time, and you are craving Italian food. So, let's define a query to search for Italian food that is within 500 metres from the Conrad Hotel. 

In [4]:
search_query = 'Italian'
radius = 500
print(search_query + ' .... OK!')

Italian .... OK!


#### Define the corresponding URL

In [5]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=534COXFXZVA22TB0B2PDNFEMGEHRNUXEHLSU5EN1BB0ICK1M&client_secret=XOUOKKHZCK1REWSLR3KYR4FQKCMWDWZVXNHSFW5BXNOQMB0V&ll=40.7149555,-74.0153365&v=20180604&query=Italian&radius=500&limit=30'

#### Send the GET Request and examine the results

In [6]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e287649006dce001b88f41f'},
 'response': {'venues': [{'id': '4fa862b3e4b0ebff2f749f06',
    'name': "Harry's Italian Pizza Bar",
    'location': {'address': '225 Murray St',
     'lat': 40.71521779064671,
     'lng': -74.01473940209351,
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.71521779064671,
       'lng': -74.01473940209351}],
     'distance': 58,
     'postalCode': '10282',
     'cc': 'US',
     'city': 'New York',
     'state': 'NY',
     'country': 'United States',
     'formattedAddress': ['225 Murray St',
      'New York, NY 10282',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d1ca941735',
      'name': 'Pizza Place',
      'pluralName': 'Pizza Places',
      'shortName': 'Pizza',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/pizza_',
       'suffix': '.png'},
      'primary': True}],
    'delivery': {'id': '294544',
     'url': 'https://www.seamless.com/menu/harrys-italian-pizz

#### Get relevant part of JSON and transform it into a *pandas* dataframe

In [7]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,...,location.state,location.country,location.formattedAddress,delivery.id,delivery.url,delivery.provider.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.icon.name,location.crossStreet
0,4fa862b3e4b0ebff2f749f06,Harry's Italian Pizza Bar,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",v-1579710166,False,225 Murray St,40.715218,-74.014739,"[{'label': 'display', 'lat': 40.71521779064671...",58,...,NY,United States,"[225 Murray St, New York, NY 10282, United Sta...",294544,https://www.seamless.com/menu/harrys-italian-p...,seamless,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_seamless_20180129.png,NaN
1,4f3232e219836c91c7bfde94,Conca Cucina Italian Restaurant,"[{'id': '4d4b7105d754a06374d81259', 'name': 'F...",v-1579710166,False,63 W Broadway,40.714484,-74.009806,"[{'label': 'display', 'lat': 40.71448400000000...",469,...,NY,United States,"[63 W Broadway, New York, NY 10007, United Sta...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3fd66200f964a520f4e41ee3,Ecco,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",v-1579710166,False,124 Chambers St,40.715337,-74.008848,"[{'label': 'display', 'lat': 40.71533713859952...",549,...,NY,United States,[124 Chambers St (btwn Church St & W Broadway)...,NaN,NaN,NaN,NaN,NaN,NaN,btwn Church St & W Broadway


#### Define information of interest and filter dataframe

In [8]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,id
0,Harry's Italian Pizza Bar,Pizza Place,225 Murray St,40.715218,-74.014739,"[{'label': 'display', 'lat': 40.71521779064671...",58,10282,US,New York,NY,United States,"[225 Murray St, New York, NY 10282, United Sta...",NaN,4fa862b3e4b0ebff2f749f06
1,Conca Cucina Italian Restaurant,Food,63 W Broadway,40.714484,-74.009806,"[{'label': 'display', 'lat': 40.71448400000000...",469,10007,US,New York,NY,United States,"[63 W Broadway, New York, NY 10007, United Sta...",NaN,4f3232e219836c91c7bfde94
2,Ecco,Italian Restaurant,124 Chambers St,40.715337,-74.008848,"[{'label': 'display', 'lat': 40.71533713859952...",549,10007,US,New York,NY,United States,[124 Chambers St (btwn Church St & W Broadway)...,btwn Church St & W Broadway,3fd66200f964a520f4e41ee3


#### Let's visualize the Italian restaurants that are nearby

In [9]:
dataframe_filtered.name

0          Harry's Italian Pizza Bar
1    Conca Cucina Italian Restaurant
2                               Ecco
Name: name, dtype: object

In [12]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.vector_layers.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Conrad Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.vector_layers.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

<a id="item2"></a>

## 2. Explore a Given Venue
> `https://api.foursquare.com/v2/venues/`**VENUE_ID**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&v=`**VERSION**

### A. Let's explore the closest Italian restaurant -- _Harry's Italian Pizza Bar_

In [13]:
venue_id = '4fa862b3e4b0ebff2f749f06' # ID of Harry's Italian Pizza Bar
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
url

'https://api.foursquare.com/v2/venues/4fa862b3e4b0ebff2f749f06?client_id=534COXFXZVA22TB0B2PDNFEMGEHRNUXEHLSU5EN1BB0ICK1M&client_secret=XOUOKKHZCK1REWSLR3KYR4FQKCMWDWZVXNHSFW5BXNOQMB0V&v=20180604'

#### Send GET request for result

In [14]:
result = requests.get(url).json()
print(result['response']['venue'].keys())
result['response']['venue']

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'url', 'price', 'hasMenu', 'likes', 'dislike', 'ok', 'rating', 'ratingColor', 'ratingSignals', 'delivery', 'menu', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'hours', 'popular', 'pageUpdates', 'inbox', 'attributes', 'bestPhoto', 'colors'])


{'id': '4fa862b3e4b0ebff2f749f06',
 'name': "Harry's Italian Pizza Bar",
 'contact': {'phone': '2126081007', 'formattedPhone': '(212) 608-1007'},
 'location': {'address': '225 Murray St',
  'lat': 40.71521779064671,
  'lng': -74.01473940209351,
  'labeledLatLngs': [{'label': 'display',
    'lat': 40.71521779064671,
    'lng': -74.01473940209351}],
  'postalCode': '10282',
  'cc': 'US',
  'city': 'New York',
  'state': 'NY',
  'country': 'United States',
  'formattedAddress': ['225 Murray St',
   'New York, NY 10282',
   'United States']},
 'canonicalUrl': 'https://foursquare.com/v/harrys-italian-pizza-bar/4fa862b3e4b0ebff2f749f06',
 'categories': [{'id': '4bf58dd8d48988d1ca941735',
   'name': 'Pizza Place',
   'pluralName': 'Pizza Places',
   'shortName': 'Pizza',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/pizza_',
    'suffix': '.png'},
   'primary': True},
  {'id': '4bf58dd8d48988d110941735',
   'name': 'Italian Restaurant',
   'pluralName': 'Italian Restauran

### B. Get the venue's overall rating

In [15]:
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

7.0


That is not a very good rating. Let's check the rating of the second closest Italian restaurant.

In [16]:
venue_id = '4f3232e219836c91c7bfde94' # ID of Conca Cucina Italian Restaurant
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


Since this restaurant has no ratings, let's check the third restaurant.

In [17]:
venue_id = '3fd66200f964a520f4e41ee3' # ID of Ecco
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

7.7


Since this restaurant has a slightly better rating, let's explore it further.

### C. Get the number of tips

In [18]:
result['response']['venue']['tips']['count']

19

### D. Get the venue's tips
> `https://api.foursquare.com/v2/venues/`**VENUE_ID**`/tips?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&v=`**VERSION**`&limit=`**LIMIT**

#### Create URL and send GET request. Make sure to set limit to get all tips

In [19]:
## Ecco Tips
limit = 15 # set limit to be greater than or equal to the total number of tips
url = 'https://api.foursquare.com/v2/venues/{}/tips?client_id={}&client_secret={}&v={}&limit={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION, limit)

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e2877d3be61c9001e45add1'},
 'response': {'tips': {'count': 19,
   'items': [{'id': '5ab1cb46c9a517174651d3fe',
     'createdAt': 1521601350,
     'text': 'A+ Italian food! Trust me on this: my mom’s side of the family is 100% Italian. I was born and bred to know good pasta when I see it, and Ecco is one of my all-time NYC favorites',
     'type': 'user',
     'canonicalUrl': 'https://foursquare.com/item/5ab1cb46c9a517174651d3fe',
     'lang': 'en',
     'likes': {'count': 0, 'groups': []},
     'logView': True,
     'agreeCount': 4,
     'disagreeCount': 0,
     'lastVoteText': 'Upvoted Dec 14, 2019',
     'lastUpvoteTimestamp': 1576365433,
     'todo': {'count': 0},
     'user': {'id': '484542633',
      'firstName': 'Nick',
      'lastName': 'E',
      'photo': {'prefix': 'https://fastly.4sqi.net/img/user/',
       'suffix': '/484542633_mK2Yum7T_7Tn9fWpndidJsmw2Hof_6T5vJBKCHPLMK5OL-U5ZiJGj51iwBstcpDLYa3Zvhvis.jpg'}},
     'authorInteractionType':

#### Get tips and list of associated features

In [20]:
tips = results['response']['tips']['items']

tip = results['response']['tips']['items'][0]
tip.keys()

dict_keys(['id', 'createdAt', 'text', 'type', 'canonicalUrl', 'lang', 'likes', 'logView', 'agreeCount', 'disagreeCount', 'lastVoteText', 'lastUpvoteTimestamp', 'todo', 'user', 'authorInteractionType'])

#### Format column width and display all tips

In [21]:
pd.set_option('display.max_colwidth', -1)

tips_df = json_normalize(tips) # json normalize tips

# columns to keep
filtered_columns = ['text', 'agreeCount', 'disagreeCount', 'id', 'user.firstName', 'user.lastName', 'user.gender', 'user.id']
tips_filtered = tips_df.loc[:, filtered_columns]

# display tips
tips_filtered

C:\Users\Jarvis\miniconda3\lib\site-packages\pandas\core\indexing.py:1418: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


,text,agreeCount,disagreeCount,id,user.firstName,user.lastName,user.gender,user.id
0,"A+ Italian food! Trust me on this: my mom’s side of the family is 100% Italian. I was born and bred to know good pasta when I see it, and Ecco is one of my all-time NYC favorites",4,0,5ab1cb46c9a517174651d3fe,Nick,E,NaN,484542633


Now remember that because we are using a personal developer account, then we can access only 2 of the restaurant's tips, instead of all 15 tips.

<a id="item3"></a>

## 3. Search a Foursquare User
> `https://api.foursquare.com/v2/users/`**USER_ID**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&v=`**VERSION**

### Define URL, send GET request and display features associated with user

In [26]:
user_id = '11830152' # user ID with most agree counts and complete profile

url = 'https://api.foursquare.com/v2/users/{}?client_id={}&client_secret={}&v={}'.format(user_id, CLIENT_ID, CLIENT_SECRET, VERSION) # define URL

# send GET request
results = requests.get(url).json()
user_data = results['response']['user']

# display features associated with user
user_data.keys()

KeyError: 'user'

In [27]:
print('First Name: ' + user_data['firstName'])
print('Last Name: ' + user_data['lastName'])
print('Home City: ' + user_data['homeCity'])

KeyError: 'firstName'

#### How many tips has this user submitted?

In [28]:
user_data['tips']

KeyError: 'tips'

Wow! So it turns out that Nick is a very active Foursquare user, with more than 250 tips.

### Get User's tips

In [29]:
# define tips URL
url = 'https://api.foursquare.com/v2/users/{}/tips?client_id={}&client_secret={}&v={}&limit={}'.format(user_id, CLIENT_ID, CLIENT_SECRET, VERSION, limit)

# send GET request and get user's tips
results = requests.get(url).json()
tips = results['response']['tips']['items']

# format column width
pd.set_option('display.max_colwidth', -1)

tips_df = json_normalize(tips)

# filter columns
filtered_columns = ['text', 'agreeCount', 'disagreeCount', 'id']
tips_filtered = tips_df.loc[:, filtered_columns]

# display user's tips
tips_filtered

,text,agreeCount,disagreeCount,id
0,"While it is far from the fastest way to travel, the ride is especially beautiful from 72nd to 135th riverside by the park.",0,0,53c08e42498ef311ea7f347a
1,"Total gem of a coffee shop,right on Broadway! Look down or you might miss it!",0,0,536c14f811d2b89770bea4fa
2,One of the the best gay dives still left in the city! Totally unpretentious!,0,0,536c1381498e14d9e19e344d
3,Happy hour is still one of the best deals in the city with wells and most beers starting @ $3!,2,0,536c12f0498e79db13f449f5
4,Most Friday's there is karaoke and an extended 2-4-1 Happy Hour!!,0,0,536c125d498e14d9e19e0b95
5,One of the best close-out/discount retail gems in the city! Be prepared to rummage a little but it's worth it because you never know what you'll find!!!,1,0,52aa362c498e058e2090a5a7
6,Great views of the Jersey City skyline and Hudson River! Especially beautiful this holiday season inside with 2 Xmas trees in an open public space and a good place to enjoy free wifi and warm drink!,6,0,52a6398a11d28c7ff30c43cd
7,Someone just got their hair did! You go girl!,0,0,4ed3f6019adf25445b0d74ad
8,Pumpkin spice latte! Get it while you can!,1,0,4ed3f496be7b9eece41c123a
9,"Try their sundaes in a word, AWSOME!",0,0,4e2b8bdd1fc71966bef06200


#### Let's get the venue for the tip with the greatest number of agree counts

In [30]:
tip_id = '5ab5575d73fe2516ad8f363b' # tip id

# define URL
url = 'http://api.foursquare.com/v2/tips/{}?client_id={}&client_secret={}&v={}'.format(tip_id, CLIENT_ID, CLIENT_SECRET, VERSION)

# send GET Request and examine results
result = requests.get(url).json()
print(result['response']['tip']['venue']['name'])
print(result['response']['tip']['venue']['location'])

Cowgirl
{'address': '519 Hudson St', 'crossStreet': 'at W 10th St', 'lat': 40.73373338282062, 'lng': -74.0062998849649, 'labeledLatLngs': [{'label': 'display', 'lat': 40.73373338282062, 'lng': -74.0062998849649}], 'postalCode': '10014', 'cc': 'US', 'city': 'New York', 'state': 'NY', 'country': 'United States', 'formattedAddress': ['519 Hudson St (at W 10th St)', 'New York, NY 10014', 'United States']}


### Get User's friends

In [31]:
user_friends = json_normalize(user_data['friends']['groups'][0]['items'])
user_friends

KeyError: 'friends'

Interesting. Despite being very active, it turns out that Nick does not have any friends on Foursquare. This might definitely change in the future.

### Retrieve the User's Profile Image

In [32]:
user_data

{}

In [33]:
# 1. grab prefix of photo
# 2. grab suffix of photo
# 3. concatenate them using the image size  
Image(url='https://igx.4sqi.net/img/user/300x300/484542633_mK2Yum7T_7Tn9fWpndidJsmw2Hof_6T5vJBKCHPLMK5OL-U5ZiJGj51iwBstcpDLYa3Zvhvis.jpg')

<a id="item4"></a>

## 4. Explore a location
> `https://api.foursquare.com/v2/venues/`**explore**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&ll=`**LATITUDE**`,`**LONGITUDE**`&v=`**VERSION**`&limit=`**LIMIT**

#### So, you just finished your gourmet dish at Ecco, and are just curious about the popular spots around the restaurant. In order to explore the area, let's start by getting the latitude and longitude values of Ecco Restaurant.

In [34]:
latitude = 40.715337
longitude = -74.008848

#### Define URL

In [35]:
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=534COXFXZVA22TB0B2PDNFEMGEHRNUXEHLSU5EN1BB0ICK1M&client_secret=XOUOKKHZCK1REWSLR3KYR4FQKCMWDWZVXNHSFW5BXNOQMB0V&ll=40.715337,-74.008848&v=20180604&radius=500&limit=30'

#### Send GET request and examine results

In [36]:
import requests

In [37]:
results = requests.get(url).json()
'There are {} around Ecco restaurant.'.format(len(results['response']['groups'][0]['items']))

'There are 30 around Ecco restaurant.'

#### Get relevant part of JSON

In [38]:
items = results['response']['groups'][0]['items']
items[0]

{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '4af5d65ff964a52091fd21e3',
  'name': 'Korin',
  'location': {'address': '57 Warren St',
   'crossStreet': 'Church St',
   'lat': 40.71482437714839,
   'lng': -74.00940425461492,
   'labeledLatLngs': [{'label': 'display',
     'lat': 40.71482437714839,
     'lng': -74.00940425461492}],
   'distance': 73,
   'postalCode': '10007',
   'cc': 'US',
   'neighborhood': 'Tribeca',
   'city': 'New York',
   'state': 'NY',
   'country': 'United States',
   'formattedAddress': ['57 Warren St (Church St)',
    'New York, NY 10007',
    'United States']},
  'categories': [{'id': '4bf58dd8d48988d1f8941735',
    'name': 'Furniture / Home Store',
    'pluralName': 'Furniture / Home Stores',
    'shortName': 'Furniture / Home',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/furniture_',
     'suffix': '.png'},
    'primar

#### Process JSON and convert it to a clean dataframe

In [39]:
dataframe = json_normalize(items) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# filter the category for each row
dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean columns
dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]

dataframe_filtered.head(10)

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,neighborhood,city,state,country,formattedAddress,id
0,Korin,Furniture / Home Store,57 Warren St,Church St,40.714824,-74.009404,"[{'label': 'display', 'lat': 40.71482437714839, 'lng': -74.00940425461492}]",73,10007,US,Tribeca,New York,NY,United States,"[57 Warren St (Church St), New York, NY 10007, United States]",4af5d65ff964a52091fd21e3
1,Chambers Street Wines,Wine Shop,148 Chambers St,btwn West Broadway & Hudson St,40.715773,-74.009718,"[{'label': 'display', 'lat': 40.715773063928374, 'lng': -74.00971823312332}]",88,10007,US,NaN,New York,NY,United States,"[148 Chambers St (btwn West Broadway & Hudson St), New York, NY 10007, United States]",4adcf23cf964a520cc6221e3
2,Takahachi Bakery,Bakery,25 Murray St,at Church St,40.713653,-74.008804,"[{'label': 'display', 'lat': 40.713652845301894, 'lng': -74.0088038953017}]",187,10007,US,NaN,New York,NY,United States,"[25 Murray St (at Church St), New York, NY 10007, United States]",4c154c9a77cea593c401d260
3,Los Tacos No. 1,Taco Place,136 Church St,NaN,40.714267,-74.008756,"[{'label': 'display', 'lat': 40.714267, 'lng': -74.008756}]",119,10007,US,NaN,New York,NY,United States,"[136 Church St, New York, NY 10007, United States]",5d5f24ec09484500079aee00
4,Heyday,Spa,92 Reade St,NaN,40.715726,-74.007767,"[{'label': 'display', 'lat': 40.715726, 'lng': -74.007767}]",100,10013,US,NaN,New York,NY,United States,"[92 Reade St, New York, NY 10013, United States]",57ad129c498e05b086594d72
5,Juice Press,Vegetarian / Vegan Restaurant,83 Murray St,btwn Greenwich St & W Broadway,40.714788,-74.011132,"[{'label': 'display', 'lat': 40.71478769908051, 'lng': -74.0111317502157}]",202,10007,US,NaN,New York,NY,United States,"[83 Murray St (btwn Greenwich St & W Broadway), New York, NY 10007, United States]",54148bc6498ea7bb8c05b70a
6,Takahachi,Sushi Restaurant,145 Duane St,btwn W Broadway & Church St,40.716526,-74.008101,"[{'label': 'display', 'lat': 40.71652647412374, 'lng': -74.00810108466207}]",146,10013,US,NaN,New York,NY,United States,"[145 Duane St (btwn W Broadway & Church St), New York, NY 10013, United States]",4a8f2f39f964a520471420e3
7,Nish Nūsh,Falafel Restaurant,88 Reade St,at Church St,40.715537,-74.007725,"[{'label': 'display', 'lat': 40.71553710116416, 'lng': -74.00772452925565}]",97,10013,US,NaN,New York,NY,United States,"[88 Reade St (at Church St), New York, NY 10013, United States]",50ba9119e4b071a4bae6dc10
8,Equinox Tribeca,Gym,54 Murray St,at W Broadway,40.714099,-74.009686,"[{'label': 'display', 'lat': 40.71409860726041, 'lng': -74.0096857179283}]",154,10007,US,NaN,New York,NY,United States,"[54 Murray St (at W Broadway), New York, NY 10007, United States]",4a6e331af964a52031d41fe3
9,Whole Foods Market,Grocery Store,270 Greenwich Street,at Warren St,40.715579,-74.011368,"[{'label': 'display', 'lat': 40.715579155420606, 'lng': -74.01136823958119}]",214,10007,US,Tribeca,New York,NY,United States,"[270 Greenwich Street (at Warren St), New York, NY 10007, United States]",49bc3b0af964a52020541fe3


#### Let's visualize these items on the map around our location

In [41]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Ecco


# add Ecco as a red circle mark
folium.vector_layers.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup='Ecco',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)


# add popular spots to the map as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.vector_layers.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(venues_map)

# display map
venues_map

<a id="item5"></a>

## 5. Explore Trending Venues
> `https://api.foursquare.com/v2/venues/`**trending**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&ll=`**LATITUDE**`,`**LONGITUDE**`&v=`**VERSION**

#### Now, instead of simply exploring the area around Ecco, you are interested in knowing the venues that are trending at the time you are done with your lunch, meaning the places with the highest foot traffic. So let's do that and get the trending venues around Ecco.

In [42]:
# define URL
url = 'https://api.foursquare.com/v2/venues/trending?client_id={}&client_secret={}&ll={},{}&v={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION)

# send GET request and get trending venues
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e2878fda2e538001b5ff03f'},
 'response': {'venues': [{'id': '58a1abe9326c5a3a1cc5ba94',
    'name': 'Spotify WTC',
    'location': {'address': '4 World Trade Ctr',
     'lat': 40.71027184019529,
     'lng': -74.01204943656921,
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.71027184019529,
       'lng': -74.01204943656921}],
     'distance': 625,
     'cc': 'US',
     'city': 'New York',
     'state': 'NY',
     'country': 'United States',
     'formattedAddress': ['4 World Trade Ctr',
      'New York, NY',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d125941735',
      'name': 'Tech Startup',
      'pluralName': 'Tech Startups',
      'shortName': 'Tech Startup',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/technology_',
       'suffix': '.png'},
      'primary': True}]}]}}

### Check if any venues are trending at this time

In [43]:
if len(results['response']['venues']) == 0:
    trending_venues_df = 'No trending venues are available at the moment!'
    
else:
    trending_venues = results['response']['venues']
    trending_venues_df = json_normalize(trending_venues)

    # filter columns
    columns_filtered = ['name', 'categories'] + ['location.distance', 'location.city', 'location.postalCode', 'location.state', 'location.country', 'location.lat', 'location.lng']
    trending_venues_df = trending_venues_df.loc[:, columns_filtered]

    # filter the category for each row
    trending_venues_df['categories'] = trending_venues_df.apply(get_category_type, axis=1)

C:\Users\Jarvis\miniconda3\lib\site-packages\pandas\core\indexing.py:1418: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


In [44]:
# display trending venues
trending_venues_df

,name,categories,location.distance,location.city,location.postalCode,location.state,location.country,location.lat,location.lng
0,Spotify WTC,Tech Startup,625,New York,NaN,NY,United States,40.710272,-74.012049


Now, depending on when you run the above code, you might get different venues since the venues with the highest foot traffic are fetched live. 

### Visualize trending venues

In [47]:
if len(results['response']['venues']) == 0:
    venues_map = 'Cannot generate visual as no trending venues are available at the moment!'

else:
    venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Ecco


    # add Ecco as a red circle mark
    folium.vector_layers.CircleMarker(
        [latitude, longitude],
        radius=10,
        popup='Ecco',
        fill=True,
        color='red',
        fill_color='red',
        fill_opacity=0.6
    ).add_to(venues_map)


    # add the trending venues as blue circle markers
    for lat, lng, label in zip(trending_venues_df['location.lat'], trending_venues_df['location.lng'], trending_venues_df['name']):
        folium.vector_layers.CircleMarker(
            [lat, lng],
            radius=5,
            poup=label,
            fill=True,
            color='blue',
            fill_color='blue',
            fill_opacity=0.6
        ).add_to(venues_map)

In [48]:
# display map
venues_map

<a id="item6"></a>

### Thank you for completing this lab!

This notebook was created by [Alex Aklson](https://www.linkedin.com/in/aklson/). I hope you found this lab interesting and educational. Feel free to contact me if you have any questions!

This notebook is part of a course on **Coursera** called *Applied Data Science Capstone*. If you accessed this notebook outside the course, you can take this course online by clicking [here](http://cocl.us/DP0701EN_Coursera_Week2_LAB1).

<hr>
Copyright &copy; 2018 [Cognitive Class](https://cognitiveclass.ai/?utm_source=bducopyrightlink&utm_medium=dswb&utm_campaign=bdu). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/).